In [68]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
service = Service(ChromeDriverManager().install())
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta

# ChromeDriverManager를 통해 드라이버 경로를 설정


options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")
driver = webdriver.Chrome(options=options)
driver.close()

In [69]:
# 채용 공고 가져오기

# Selenium WebDriver 설정
driver = webdriver.Chrome() 

# 결과를 저장할 리스트
company_name = [] # 회사명
title = [] # 공고 제목
education = [] # 학력
career = [] # 경력
dead_line = [] # 마감일
skill = [] # 기술 스택

# 채용 공고를 낸 회사 정보 저장할 리스트
company_place = [] # 회사 위치
company_category = [] # 업종
company_url = [] # 홈페이지



# 날짜 데이터 전처리
def convert_deadline(deadline_text):
    try:
        # D-_일 형태
        if 'D-' in deadline_text:
            days_remaining = int(deadline_text.split('D-')[1].split('일')[0])
            return (datetime.now() + timedelta(days=days_remaining)).strftime('%Y.%m.%d')

        # ~__.__(요일) 형태
        elif '~' in deadline_text:
            date_str = deadline_text.split('~')[1].split('(')[0].strip()  
            date_obj = datetime.strptime(date_str, '%m.%d')  
            return date_obj.replace(year=datetime.now().year).strftime('%Y.%m.%d')

        # 내일마감
        elif '내일마감' in deadline_text:
            return (datetime.now() + timedelta(days=1)).strftime('%Y.%m.%d')

        # 채용시
        elif '채용시' in deadline_text:
            return None
        
        elif '상시채용' in deadline_text:
            return None

        return deadline_text

    except Exception as e:
        print(f"마감일 변환 중 오류 발생: {e}")
        return deadline_text  # 오류가 발생하면 원본 텍스트 반환


# 페이지 넘기면서
for i in range(1, 4):
    try:

        # 페이지 URL
        url = f"https://www.saramin.co.kr/zf_user/jobs/list/job-category?page={i}&cat_mcls=2&search_optional_item=n&search_done=y&panel_count=y&preview=y&isAjaxRequest=0&page_count=50&sort=RL&type=job-category&is_param=1&isSearchResultEmpty=1&isSectionHome=0&searchParamCount=1#searchTitle"
        driver.get(url)  # 페이지 열기
        time.sleep(2)  # 벤 안먹기 위한 슬립
        current_url = driver.current_url

        # 페이지 소스를 BeautifulSoup으로 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 회사 이름이 하이퍼 링크인 경우만 회사 정보 & 채용 공고 가져오기
        # 회사 이름과 채용 공고 제목 추출
        atagData = soup.find_all(['a'], class_='str_tit')  # 'a'와 'span' 태그에서 class='str_tit'을 가진 모든 요소 찾기

        for company in atagData:
    
            if company.name == 'a' and 'jobs/relay' in company.get('href', ''):
                title.append(company.get_text(strip=True))
    
            # 'a' 태그인 경우 href 속성을 통해 회사 이름과 채용 공고 제목을 구분
            if company.name == 'a' and 'company-info' in company.get('href', ''):  
                # 회사 이름 조건                               # 채용 공고 제목 조건

                
                company_name.append(company.get_text(strip=True))
                company_href = company.get('href')
                href = "https://www.saramin.co.kr" + company_href

                driver.get(href)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')

                base_url = 'https://www.saramin.co.kr/zf_user/company-info/view?csn='
                selected_company = driver.current_url
                tmp = selected_company[70:]
                time.sleep(2)
                
                # 기업소개 버튼
                move_to_company_descript= base_url + tmp

                # 
                driver.get(move_to_company_descript)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(2)

                # 기업 데이터 가져오기
                company_data = soup.select("dl.company_details > div.company_details_group > dd.desc")

                # 회사 데이터 추출
                if len(company_data) > 0:
                    # 1번째 데이터 (업종 정보)
                    company_category.append(company_data[0].get_text(strip=True) if company_category else "No category")
                    
                else:
                    company_category.append("No category")

                if len(company_data) > 2:
                    # 3번째 데이터 (홈페이지 URL)
                    homepage_tag = company_data[2].find("a", target="_blank")
                    company_url.append(homepage_tag["href"] if homepage_tag else "No homepage URL")
                else:
                    company_url.append("No hompage URL")

                if len(company_data) > 4:
                    # 5번째 데이터 (주소 정보)
                    address_tag = company_data[4].find("p", class_="ellipsis")
                    company_place.append(address_tag.get_text(strip=True) if address_tag else "No address") 
                else:
                    company_place.append("No address")

                     
                
            time.sleep(2)   

        driver.get(current_url)
        time.sleep(1)
        # 다시 읽어오기    
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(2)
        

    
        # 'span' 태그인 경우에도 텍스트 추출
        if company.name == 'span':  
            continue # 하이퍼 링크가 아니면 그냥 데이터를 추출하지 않음
    
        # 기술 스택 추출
        sData = soup.find_all('div', class_='job_meta')
        
        for skilldata in sData:
            # job_sector 클래스 내부의 span 태그 추출
            sector_spans = skilldata.find_all('span', class_='job_sector')

            # 5개씩 한 묶음
            single_skill_set = [] 
            for sector in sector_spans:
                # 클래스가 없는 내부 span 태그 추출
                inner_spans = sector.find_all('span', class_=None)
                for span in inner_spans:
                    single_skill_set.append(span.get_text(strip=True))

            if single_skill_set:
                skill.append(",".join(single_skill_set))        

        # education, career, postdate, deadline 추출
        education_tags = soup.find_all('p', class_='education')
        career_tags = soup.find_all('p', class_='career')
        deadline_tags = soup.find_all('p', class_='support_detail')

        for tag in deadline_tags:
            span_tag = tag.find('span', class_='date')
            if span_tag:
                dead_line.append(convert_deadline(span_tag.get_text(strip=True)) if span_tag else "마감 일 입력되지 않음") # 마감일 정규화

        # 저장
        for tag in education_tags:
            education.append(tag.get_text(strip=True) if tag else "학력 사항 입력되지 않음")
        for tag in career_tags:
            career.append(tag.get_text(strip=True) if tag else "경력 사항 입력되지 않음")

    except Exception as e:
         print(f"에러 발생: {e}")

#driver.close()




In [70]:
# 기업 정보 가져오기


from selenium import webdriver
from bs4 import BeautifulSoup
import time

# WebDriver 설정
driver = webdriver.Chrome()

# 결과 저장 리스트
curation_company_name = []  # 기업명
curation_company_type = []  # 기업 형태
curation_company_year = []  # 기업 설립 연도
curation_company_genre = []  # 기업 업종

# 초기 URL 설정
url = 'https://www.saramin.co.kr/zf_user/company-info/sri-certification?seq=236'
driver.get(url)
time.sleep(2)

# 페이지 버튼 누르면서 반복
for page in range(1, 6): 
    try:
        # 페이지 소스 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 큐레이션 리스트에 있는 li 탐색
        ul_tags = soup.findAll('ul', class_='list_employ list_company')

        for ul_tag in ul_tags:
            # 회사명 태그 찾기
            name_tags = ul_tag.find_all('a', class_='tit')
            for name_tag in name_tags:
                curation_company_name.append(name_tag.get_text(strip=True))

            # 회사 설명 태그 찾기    
            state_tags = ul_tag.find_all('div', class_='state')
            for state_tag in state_tags:
                spans = state_tag.find_all('span')
                if len(spans) >= 3:
                    # 각 항목이 비어 있으면 '입력되지 않음' 추가
                    curation_company_type.append(spans[0].get_text(strip=True) if spans[0].get_text(strip=True) else "입력되지 않음")
                    curation_company_year.append(spans[1].get_text(strip=True) if spans[1].get_text(strip=True) else "입력되지 않음")
                    curation_company_genre.append(spans[2].get_text(strip=True) if spans[2].get_text(strip=True) else "입력되지 않음")
                else:
                    # span 태그가 부족할 경우 '입력되지 않음' 추가
                    curation_company_type.append("입력되지 않음")
                    curation_company_year.append("입력되지 않음")
                    curation_company_genre.append("입력되지 않음")

        # 다음 페이지 버튼 클릭
        page_link = driver.find_element(By.CSS_SELECTOR, f'a.page[data-page="{page + 1}"]')
        page_link.click()
        time.sleep(2)  # 페이지 이동 후 대기
        
    except Exception as e:
        print(f"페이지 {page} 처리 중 에러 발생: {e}")
        break

#driver.close()


In [71]:
import csv
from itertools import zip_longest

# 데이터 예시 (리스트가 동일한 길이라고 가정)

posting_company_data = {
    "company_name" : company_name,
    "company_category" : company_category,
    "company_place" : company_place,
    "company_url" : company_url,
}

posting_data = {
    "company_name": company_name,
    "title": title,
    "education": education,
    "career": career,
    "deadline": dead_line,
    "skill": skill,
    
}

curation_company_data = {
    "curation_company_name": curation_company_name,
    "curation_company_type": curation_company_type,
    "curation_company_year": curation_company_year,
    "curation_company_genre": curation_company_genre,
}


# CSV 파일 저장
with open('data.csv', 'w', encoding='utf-8', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # CSV 파일의 헤더 작성
    writer.writerow(posting_data.keys())
    

    # 데이터를 행 단위로 작성 (zip 사용)
    p_rows = zip_longest(*posting_data.values(), fillvalue='')
    writer.writerows(p_rows)
    
with open('company_data.csv','w',encoding='utf-8', newline='') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(curation_company_data.keys())

    c_rows = zip_longest(*curation_company_data.values(), fillvalue='')
    writer.writerows(c_rows)


with open('posting_company_data.csv', 'w', encoding='utf-8', newline='') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(posting_company_data.keys())

    b_row = zip_longest(*posting_company_data.values(), fillvalue='')
    writer.writerows(b_row)



In [72]:
import pymysql
import pandas as pd

# DB 정보
host = "localhost"
user = "root"
password = "1234"
database = "job_posting"

pos_com_data = pd.read_csv('posting_company_data.csv')
pos_data = pd.read_csv('data.csv')
curation_data = pd.read_csv('company_data.csv')

# DB 연결된 상태인지 확인하기 위한 try
try:
    conn = pymysql.connect(host=host, user=user, password=password, db=database)
    print("DB 연결 성공!")
    curs = conn.cursor(pymysql.cursors.DictCursor)

    # 공고 회사 데이터 삽입
    for index, row in pos_com_data.iterrows():
        try:
            # 중복 확인
            check_sql = """
            SELECT COUNT(*) FROM Company WHERE company_name = %s
            """
            curs.execute(check_sql, (row['company_name'],))
            result = curs.fetchone()
            if result['COUNT(*)'] == 0:  # 중복이 없을 경우
                sql = """
                INSERT INTO Company (company_name, company_category, company_place, company_url)
                VALUES (%s, %s, %s, %s)
                """
                values = (
                    row['company_name'],
                    row['company_category'],
                    row['company_place'],
                    row['company_url']
                )
                curs.execute(sql, values)
                conn.commit()
            else:
                print()#f"중복된 회사명: {row['company_name']}")

        except Exception as e:
            print(f"1. 오류 발생: {e}")
            conn.rollback()

    # 공고 정보 삽입 부분 수정
    for index, row in pos_data.iterrows():
        try:
            # 회사명을 기반으로 company_id 조회
            find_name_sql = """
            SELECT company_id FROM Company WHERE company_name = %s
            """
            curs.execute(find_name_sql, (row['company_name'],))
            company = curs.fetchone()
    
            if company:
                company_id = company['company_id']
                # 중복 체크
                check_sql = """
                SELECT COUNT(*) FROM Posting WHERE company_id = %s AND title = %s
                """
                curs.execute(check_sql, (company_id, row['title']))
                result = curs.fetchone()
    
                if result['COUNT(*)'] == 0:  # 중복되지 않으면 삽입
                    # 공고 정보 삽입
                    sql = """
                    INSERT INTO Posting (company_id, title, career, education, deadline, skill)
                    VALUES (%s, %s, %s, %s, %s, %s)
                    """
                    values = (
                        company_id,
                        row['title'],
                        row['career'],
                        row['education'],
                        row['deadline'],
                        row['skill']
                    )
                    curs.execute(sql, values)
                    conn.commit()
                else:
                    print(f"중복된 공고 제목: {row['title']}")
            else:
                print(f"회사를 찾을 수 없음: {row['company_name']}")
    
        except Exception as e:
            print(f"2. 오류 발생 {e}")
            conn.rollback()


    # 대기업 정보 삽입
    for index, row in curation_data.iterrows():
        try:
            # 중복 확인
            check_sql = """
            SELECT COUNT(*) FROM Top WHERE curation_company_name = %s
            """
            curs.execute(check_sql, (row['curation_company_name'],))
            result = curs.fetchone()
            if result['COUNT(*)'] == 0:  # 중복이 없을 경우
                sql = """
                INSERT INTO Top (curation_company_name, curation_company_type, curation_company_year, curation_company_genre)
                VALUES (%s, %s, %s, %s)
                """
                values = (
                    row['curation_company_name'],
                    row['curation_company_type'],
                    row['curation_company_year'],
                    row['curation_company_genre']
                )
                curs.execute(sql, values)
                conn.commit()
            else:
                print()#f"중복된 대기업: {row['curation_company_name']}")

        except Exception as e:
            print(f"3. 오류 발생 {e}")
            conn.rollback()

except pymysql.MySQLError as e:
    print(f"DB 연결 실패: {e}")



finally:
    if 'conn' in locals() and conn.open:
        conn.close()
        print("DB 연결 닫힘.")


DB 연결 성공!







1. 오류 발생: (1406, "Data too long for column 'company_url' at row 1")
























2. 오류 발생 nan can not be used with MySQL
회사를 찾을 수 없음: (주)영신디엔씨
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
2. 오류 발생 nan can not be used with MySQL
DB 연결 닫힘.
